In [ ]:

from tabulate import tabulate
import pandas as pd
   #
pd.set_option("display.width", 480)

print("--")


In [ ]:

from katana import remote
from katana.remote import analytics

my_client = remote.Client()

print(my_client)


In [ ]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"


l_cntr = 20


print("--")


In [ ]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)

In [ ]:
######################################################

In [ ]:

#  graph.project() is used to create a homogenous group, from a heterogeneous one,
#  necessary to support some analytics
#
#  Here we'll add stuff to the graph to make it heterogeneous.


l_query = """
   CREATE ( n: Restaurant { restaurant_code: 'PAP' } )               //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.airport_code     = 'SJC'
   AND   m.restaurant_code  = 'PAP'
   CREATE (n) -[r: ATTRACTION { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


In [ ]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


In [ ]:
######################################################

In [ ]:

my_projection = my_graph.project(node_types=['Airport'], edge_types=['FLIES_TO'])

print("--")


In [ ]:


l_result1 = my_projection.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_projection.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))

#     OperationError: HaMw4a3njMMSvgQSnd2RL7r15TaavDnSvm1JePSRUjnh-5YoXGVdvYnT1L1qB5 Projected views are not supported for query operation yet. (QueryOperation.cpp:27): not implemented: not implemented



In [ ]:

#  Generate a unique prop name so we can run this over and over without edit ..
#
l_cntr += 1
   #
l_output_propname     = "betweenness_" + str(l_cntr).zfill(4)


l_result = analytics.betweenness_centrality(my_projection, result_property_name=l_output_propname, num_sources=1000)

print(l_result)



In [ ]:

l_result1 = my_graph.query(
   """
   MATCH (n) 
   RETURN n
   """ )
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query(
   """
   MATCH (n) - [r] -> (m)
   RETURN r
   """ )
print(tabulate(l_result2, headers='keys', tablefmt='psql'))


#  Sample output,
#
#     +----+-----------------+----------------+-----------+------------------+------------------+----------------------+----------+---------------------+-----------------------------+
#     |    |   n.internal_id | n.labels       | n.LABEL   | n.airport_code   | n.airport_name   |   n.betweenness_0022 | n.type   | n.restaurant_code   | n.restaurant_name           |
#     |----+-----------------+----------------+-----------+------------------+------------------+----------------------+----------+---------------------+-----------------------------|
#     |  0 |               0 | ['Airport']    | Airport   | ORD              | Chicago O-Hare   |                    4 | node     | nan                 | nan                         |
#     |  1 |               1 | ['Airport']    | Airport   | DEN              | Denver           |                    2 | node     | nan                 | nan                         |
#     |  2 | 281474976710657 | ['Restaurant'] | nan       | nan              | nan              |                  nan | node     | PAP                 | Pappadeauxs Seafood Kitchen |
#     |  3 | 281474976710656 | ['Airport']    | Airport   | SJC              | San Jose         |                    0 | node     | nan                 | nan                         |
#     |  4 | 562949953421312 | ['Airport']    | Airport   | MKE              | Milwaukee        |                    0 | node     | nan                 | nan                         |
#     +----+-----------------+----------------+-----------+------------------+------------------+----------------------+----------+---------------------+-----------------------------+
#     
#     +----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+
#     |    |        r.end.id | r.end.labels   |   r.internal_id | r.label    |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |      r.start.id | r.start.labels   | r.type   | r.DRIVABLE   |
#     |----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------|
#     |  0 |               1 | ['Airport']    |               1 | FLIES_TO   |          886 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
#     |  1 | 562949953421312 | ['Airport']    |               3 | FLIES_TO   |           66 |            1 | FLIES_TO |               0 | ['Airport']      | edge     | nan          |
#     |  2 |               0 | ['Airport']    |               0 | FLIES_TO   |          886 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
#     |  3 | 281474976710656 | ['Airport']    |               2 | FLIES_TO   |          948 |            1 | FLIES_TO |               1 | ['Airport']      | edge     | nan          |
#     |  4 |               0 | ['Airport']    | 281474976710656 | FLIES_TO   |         1829 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     | nan          |
#     |  5 |               1 | ['Airport']    | 281474976710657 | FLIES_TO   |          948 |            1 | FLIES_TO | 281474976710656 | ['Airport']      | edge     | nan          |
#     |  6 | 281474976710657 | ['Restaurant'] | 281474976710658 | ATTRACTION |          nan |          nan | nan      | 281474976710656 | ['Airport']      | edge     | Yes          |
#     |  7 |               0 | ['Airport']    | 562949953421312 | FLIES_TO   |           66 |            1 | FLIES_TO | 562949953421312 | ['Airport']      | edge     | nan          |
#     +----+-----------------+----------------+-----------------+------------+--------------+--------------+----------+-----------------+------------------+----------+--------------+


In [31]:

l_result = my_graph.query(f"""
    MATCH (n) - [r] -> (m)
    RETURN n, r, m
    """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]